# ZenPal — Demo simple

Bienvenido 👋  
Esta notebook muestra, de forma muy fácil, cómo **ZenPal**:
- Genera mensajes de apoyo (Texto → Texto).
- Crea imágenes motivacionales (Texto → Imagen).
- Calcula el costo en tiempo real según el texto generado.
- Compara **modelos** con tablas claras.

# ZenPal – Acompañante Digital para el Bienestar
Demostración de Fast Prompting utilizando:
- Texto → Texto (generación de consejos motivacionales)
- Texto → Imagen (generación de recursos visuales con gpt-image-1)

💡 Solo necesitas colocar tu **API key** antes de ejecutar las celdas y puedes **editar fácilmente** el texto (`usuario_input`) y el prompt (`prompt_imagen`) en las primeras líneas para probar otros casos.



In [2]:
# Importante para el cliente
# Cambia el texto de esta variable para probar diferentes situaciones
usuario_input = "No puedo dormir porque mañana presento un informe importante. ¿Qué hago?"
print("💡 Texto actual para ZenPal:", usuario_input)


💡 Texto actual para ZenPal: No puedo dormir porque mañana presento un informe importante. ¿Qué hago?


In [12]:
!pip install --upgrade openai


In [3]:

# --- Configuración básica (elegí modelo una sola vez) ---
import os, math, random

# 1) Tu API Key (opcional). Si no la cargás, la demo funciona igual con simulación.
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")  # o pegala acá como string

# 2) Activar pruebas reales con la API (True/False)
USE_API = False  # Cambiá a True si querés probar en vivo con tu API Key

# 3) Modelos de TEXTO disponibles para comparar
#    (Por defecto dejamos la serie GPT-5; 4.1 en la página es "optimización")
INCLUDE_GPT41 = False
MODELOS_TEXTO = ["gpt-5", "gpt-5-mini", "gpt-5-nano"]
if INCLUDE_GPT41:
    MODELOS_TEXTO.append("gpt-4.1")  # mostrará precios de optimización

# 4) Elegí el modelo por defecto para Texto→Texto y cómo comparar en tablas
modelo_texto = "gpt-5-mini"         # usado en Celda 3 y 7
MODELOS_PRUEBA = MODELOS_TEXTO[:]   # usados en Celda 5 y 6

# 5) Modelo y calidad para IMÁGENES
modelo_imagen = "gpt-image-1"
calidad_img = "image_medium"  # opciones: image_low, image_medium, image_high

# 6) Precios oficiales (OpenAI pricing) — simplificado para inferencia
PRICING = {
    # Serie GPT-5 (inferencia)
    "gpt-5": {
        "input_per_1k": 1.250,
        "input_cache_per_1k": 0.125,
        "output_per_1k": 10.000,
    },
    "gpt-5-mini": {
        "input_per_1k": 0.250,
        "input_cache_per_1k": 0.025,
        "output_per_1k": 2.000,
    },
    "gpt-5-nano": {
        "input_per_1k": 0.050,
        "input_cache_per_1k": 0.005,
        "output_per_1k": 0.400,
    },

    # GPT-4.1 (valores publicados bajo "optimización", no inferencia)
    "gpt-4.1": {
        "input_per_1k": 3.00,
        "input_cache_per_1k": 0.75,
        "output_per_1k": 12.00,
    },

    # Imágenes (GPT-image-1)
    # Además de tarifas por tokens, mostramos costo por imagen cuadrada aprox:
    "gpt-image-1": {
        "input_per_1k": 5.00,
        "input_cache_per_1k": 1.25,
        "output_per_1k": 40.00,
        "image_low": 0.01,
        "image_medium": 0.04,
        "image_high": 0.17
    }
}

# 7) Helpers muy simples
def approx_tokens(text: str) -> int:
    """Aproximación rápida: ~4 caracteres ≈ 1 token."""
    return max(1, math.ceil(len(text)/4))

def cost_text(model: str, input_text: str, output_text: str, cached_input: bool=False) -> float:
    """Calcula el costo para texto con opción de entrada cacheada."""
    prices = PRICING[model]
    pin = approx_tokens(input_text)
    pout = approx_tokens(output_text)
    in_key = "input_cache_per_1k" if cached_input and "input_cache_per_1k" in prices else "input_per_1k"
    return (pin/1000)*prices[in_key] + (pout/1000)*prices["output_per_1k"]

def cost_image(quality_key: str) -> float:
    return PRICING["gpt-image-1"][quality_key]

# 8) Material de demo sin costo (aleatorio)
PROMPTS_RANDOM = [
    "Estoy bloqueado con una tarea y no sé por dónde empezar.",
    "Mañana tengo una presentación y estoy muy nervioso.",
    "Tuve un día pesado y me cuesta concentrarme para estudiar.",
    "Quiero mejorar mi rutina sin sentirme abrumado."
]
RESPUESTAS_RANDOM = [
    "Hacé una lista con 3 pasos mínimos y arrancá por el más simple.",
    "Poné un temporizador de 20 minutos sin distracciones y movete 2 al terminar.",
    "Respirá profundo 60 segundos y acomodá tu espacio antes de retomar.",
    "Dividí la tarea en bloques chicos y celebrá cada mini avance."
]

print("✅ Config listo.")
print("   - Texto por defecto:", modelo_texto)
print("   - Modelos en tablas:", MODELOS_PRUEBA)
print("   - Imagen/calidad:", modelo_imagen, "/", calidad_img)
print("   - USE_API:", USE_API, "| API Key cargada:", bool(OPENAI_API_KEY))


✅ Config listo.
   - Texto por defecto: gpt-5-mini
   - Modelos en tablas: ['gpt-5', 'gpt-5-mini', 'gpt-5-nano']
   - Imagen/calidad: gpt-image-1 / image_medium
   - USE_API: False | API Key cargada: False


In [4]:

# --- Texto→Texto (respuesta aleatoria + costo en tiempo real) ---

# Fallback suave por si no corriste la Celda 2
import math, random
if "PRICING" not in globals():
    PRICING = {"gpt-5-mini": {"input_per_1k": 0.25, "output_per_1k": 2.00}}
if "approx_tokens" not in globals():
    def approx_tokens(text: str) -> int: return max(1, math.ceil(len(text)/4))
if "cost_text" not in globals():
    def cost_text(model: str, input_text: str, output_text: str, cached_input: bool=False) -> float:
        p = PRICING.get(model, PRICING["gpt-5-mini"])
        pin = approx_tokens(input_text); pout = approx_tokens(output_text)
        return (pin/1000)*p.get("input_per_1k",0.25) + (pout/1000)*p.get("output_per_1k",2.0)
if "PROMPTS_RANDOM" not in globals():
    PROMPTS_RANDOM = ["Necesito concentrarme mejor.", "Estoy nervioso por una entrega."]
if "RESPUESTAS_RANDOM" not in globals():
    RESPUESTAS_RANDOM = ["Hacé una lista breve y arrancá por lo más simple."]
if "modelo_texto" not in globals():
    modelo_texto = "gpt-5-mini"

system_prompt = "Sos un asistente empático que da consejos simples y concretos."
user_prompt = random.choice(PROMPTS_RANDOM)
respuesta_simulada = random.choice(RESPUESTAS_RANDOM)

# Costo estimado (sin API) según longitud real de input/output
estimated_cost = cost_text(modelo_texto, system_prompt + user_prompt, respuesta_simulada)

print("📝 Prompt elegido:", user_prompt)
print("🤖 Respuesta simulada:", respuesta_simulada)
print(f"💰 Costo estimado con {modelo_texto}: USD {estimated_cost:.6f}")

# --- Prueba REAL con la API (opcional) ---
if 'USE_API' in globals() and USE_API and 'OPENAI_API_KEY' in globals() and OPENAI_API_KEY:
    try:
        import openai
        openai.api_key = OPENAI_API_KEY

        completion = openai.ChatCompletion.create(
            model=modelo_texto,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ],
            temperature=0.7,
            max_tokens=150,
        )
        answer = completion["choices"][0]["message"]["content"]
        usage = completion.get("usage", {})
        pin = usage.get("prompt_tokens", approx_tokens(system_prompt + user_prompt))
        pout = usage.get("completion_tokens", approx_tokens(answer))
        prices = PRICING[modelo_texto]
        real_cost = (pin/1000)*prices["input_per_1k"] + (pout/1000)*prices["output_per_1k"]

        print("\n✅ API (en vivo)")
        print("🤖 Respuesta:", answer)
        print(f"🔢 Tokens → input: {pin}, output: {pout}, total: {pin+pout}")
        print(f"💰 Costo real con {modelo_texto}: USD {real_cost:.6f}")
    except Exception as e:
        print("⚠️ No se pudo usar la API (revisá tu key o librería). Error:", e)
else:
    print("ℹ️ API desactivada (USE_API=False) o falta OPENAI_API_KEY.")


📝 Prompt elegido: Estoy bloqueado con una tarea y no sé por dónde empezar.
🤖 Respuesta simulada: Respirá profundo 60 segundos y acomodá tu espacio antes de retomar.
💰 Costo estimado con gpt-5-mini: USD 0.041500
ℹ️ API desactivada (USE_API=False) o falta OPENAI_API_KEY.


In [5]:

# --- Celda 4: Texto→Imagen (costo por calidad + opción en vivo) ---

# Fallback por si no corriste Celda 2
if "cost_image" not in globals():
    def cost_image(q): return {"image_low":0.01,"image_medium":0.04,"image_high":0.17}[q]
if "calidad_img" not in globals():
    calidad_img = "image_medium"
if "modelo_imagen" not in globals():
    modelo_imagen = "gpt-image-1"

prompt_img = "Póster motivacional que diga: 'Descansar también es parte del progreso', estilo cálido y simple."
costo_img = cost_image(calidad_img)

print("🖼️ Prompt de la imagen:", prompt_img)
print(f"💰 Costo estimado ({modelo_imagen} / {calidad_img}): USD {costo_img:.2f}")

# --- Prueba REAL con la API (opcional) ---
if 'USE_API' in globals() and USE_API and 'OPENAI_API_KEY' in globals() and OPENAI_API_KEY:
    try:
        import openai
        openai.api_key = OPENAI_API_KEY
        # Descomentar si querés generar de verdad (puede tener costo):
        # resp = openai.Image.create(prompt=prompt_img, n=1, size="1024x1024")
        # print("🔗 URL imagen:", resp["data"][0]["url"])
        print("ℹ️ Llamada real comentada para evitar costes accidentales. Descomentá si querés generar.")
    except Exception as e:
        print("⚠️ No se pudo usar la API de imágenes. Error:", e)
else:
    print("ℹ️ API desactivada (USE_API=False) o falta OPENAI_API_KEY.")


🖼️ Prompt de la imagen: Póster motivacional que diga: 'Descansar también es parte del progreso', estilo cálido y simple.
💰 Costo estimado (gpt-image-1 / image_medium): USD 0.04
ℹ️ API desactivada (USE_API=False) o falta OPENAI_API_KEY.


In [6]:

# ---  Precios por modelo (USD / 1k tokens) ---
import pandas as pd

rows = []
for m in MODELOS_PRUEBA:
    p = PRICING[m]
    rows.append([m, p["input_per_1k"], p["output_per_1k"]])

df_precios = pd.DataFrame(rows, columns=["Modelo", "Entrada (USD/1k)", "Salida (USD/1k)"])
df_precios


,Modelo,Entrada (USD/1k),Salida (USD/1k)
0,gpt-5,1.25,10.0
1,gpt-5-mini,0.25,2.0
2,gpt-5-nano,0.05,0.4


In [7]:

# ---— Mismo prompt en varios modelos ---
import pandas as pd, random

system_prompt2 = "Sos un asistente empático y claro."
PROMPTS_RANDOM = PROMPTS_RANDOM if "PROMPTS_RANDOM" in globals() else [
    "Estoy bloqueado con una tarea y no sé por dónde empezar.",
    "Mañana tengo una presentación y estoy muy nervioso."
]
RESPUESTAS_RANDOM = RESPUESTAS_RANDOM if "RESPUESTAS_RANDOM" in globals() else [
    "Hacé una lista con 3 pasos mínimos y arrancá por el más simple."
]
test_prompt = random.choice(PROMPTS_RANDOM)

def run_sim(model_name: str, user_text: str):
    sim_answer = random.choice(RESPUESTAS_RANDOM)
    est_cost = cost_text(model_name, system_prompt2 + user_text, sim_answer)
    return sim_answer, approx_tokens(system_prompt2 + user_text), approx_tokens(sim_answer), est_cost

def run_live(model_name: str, user_text: str):
    try:
        import openai
        if not OPENAI_API_KEY:
            raise RuntimeError("Falta OPENAI_API_KEY")
        openai.api_key = OPENAI_API_KEY
        completion = openai.ChatCompletion.create(
            model=model_name,
            messages=[{"role": "system", "content": system_prompt2},
                      {"role": "user", "content": user_text}],
            temperature=0.7,
            max_tokens=150,
        )
        answer = completion["choices"][0]["message"]["content"]
        usage = completion.get("usage", {})
        pin  = usage.get("prompt_tokens",  approx_tokens(system_prompt2 + user_text))
        pout = usage.get("completion_tokens", approx_tokens(answer))
        prices = PRICING[model_name]
        real_cost = (pin/1000)*prices["input_per_1k"] + (pout/1000)*prices["output_per_1k"]
        return answer, pin, pout, real_cost
    except Exception as e:
        return f"(Error API: {e})", 0, 0, 0.0

rows = []
for m in MODELOS_PRUEBA:
    if USE_API and OPENAI_API_KEY:
        ans, pin, pout, cost = run_live(m, test_prompt)
    else:
        ans, pin, pout, cost = run_sim(m, test_prompt)
    rows.append([m, test_prompt, ans, pin, pout, cost])

df_compare = pd.DataFrame(rows, columns=["Modelo", "Prompt", "Respuesta", "Tokens in", "Tokens out", "Costo (USD)"])
df_compare


,Modelo,Prompt,Respuesta,Tokens in,Tokens out,Costo (USD)
0,gpt-5,Tuve un día pesado y me cuesta concentrarme pa...,Dividí la tarea en bloques chicos y celebrá ca...,23,16,0.18875
1,gpt-5-mini,Tuve un día pesado y me cuesta concentrarme pa...,Dividí la tarea en bloques chicos y celebrá ca...,23,16,0.03775
2,gpt-5-nano,Tuve un día pesado y me cuesta concentrarme pa...,Hacé una lista con 3 pasos mínimos y arrancá p...,23,16,0.00755


In [8]:

# --- Variantes aleatorias con un modelo ---
import pandas as pd, random

N = 3  # cantidad de ejecuciones
modelo = modelo_texto  # usa el modelo elegido en la Celda 2

def run_once(model_name: str):
    up = random.choice(PROMPTS_RANDOM)
    if USE_API and OPENAI_API_KEY:
        # Reutilizamos run_live de la celda anterior
        ans, pin, pout, cost = run_live(model_name, up)
    else:
        ans, pin, pout, cost = run_sim(model_name, up)
    return {"Prompt": up, "Respuesta": ans, "Tokens in": pin, "Tokens out": pout, "Costo (USD)": cost}

df_runs = pd.DataFrame([run_once(modelo) for _ in range(N)])
df_runs


,Prompt,Respuesta,Tokens in,Tokens out,Costo (USD)
0,Estoy bloqueado con una tarea y no sé por dónd...,Respirá profundo 60 segundos y acomodá tu espa...,23,17,0.03975
1,Tuve un día pesado y me cuesta concentrarme pa...,Respirá profundo 60 segundos y acomodá tu espa...,23,17,0.03975
2,Estoy bloqueado con una tarea y no sé por dónd...,Poné un temporizador de 20 minutos sin distrac...,23,19,0.04375


In [1]:
from openai import OpenAI
import os, json
from IPython.display import display, Markdown

# 🔑 API key (o setear OPENAI_API_KEY en variables de entorno antes de ejecutar)
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY", "tu_api_va_aqui")
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

# 📝 Entrada del usuario (puedes editar esta línea para probar otras situaciones)
usuario_input = "No puedo dormir porque mañana presento un informe importante. ¿Qué hago?"
print("💡 Texto actual para ZenPal:", usuario_input)

# 🎯 Fast Prompting con creatividad controlada
system_rules = """
Eres ZenPal, un acompañante digital empático y práctico.
Sigue estas reglas:
- Devuelve un JSON con las claves {"title": str, "steps": [str, str], "tone": str}.
- "title": breve y motivador, 5–7 palabras.
- "steps": exactamente dos acciones concretas, claras y realistas.
- "tone": siempre "warm".
- Sé amable, directo y motivador.
"""

# ✅ Aquí usamos la variable usuario_input
messages = [
    {"role": "system", "content": system_rules},
    {"role": "user", "content": usuario_input}  # 👈 aquí usamos la variable
]

response = client.chat.completions.create(
    model="gpt-4o",  # o gpt-4o-mini si prefieres más económico
    messages=messages,
    temperature=0.8,  # mayor creatividad = respuestas variadas
    top_p=1,
)

# Mostrar como Markdown
raw = response.choices[0].message.content

try:
    data = json.loads(raw)
    md = f"### {data['title']}\n\n**Paso 1:** {data['steps'][0]}\n\n**Paso 2:** {data['steps'][1]}\n\n_Tono: {data['tone']}._"
    display(Markdown(md))
except:
    print("Respuesta del modelo:\n")
    print(raw)



💡 Texto actual para ZenPal: No puedo dormir porque mañana presento un informe importante. ¿Qué hago?
Respuesta del modelo:

```json
{
  "title": "Relájate y Prepárate para Triunfar",
  "steps": [
    "Haz una lista de verificación rápida de tus puntos clave para el informe, asegurándote de que todo esté en su lugar.",
    "Realiza una breve meditación guiada o ejercicios de respiración para calmar tu mente y cuerpo antes de dormir."
  ],
  "tone": "warm"
}
```


In [ ]:
from openai import OpenAI
import base64, os
from IPython.display import Image, display

# 🔑 API key (o setear OPENAI_API_KEY en variables de entorno antes de ejecutar)
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY", "tu_clave_Api_va_aqui")
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

# 📝 Prompt de imagen (edita esta línea para generar algo distinto)
prompt_imagen = "Minimal breathing guide: a soft circle expanding/contracting in calm blue/green, neutral background, flat style, smooth gradients, no text."
print("🎨 Prompt actual:", prompt_imagen)

# Generar imagen
result = client.images.generate(
    model="gpt-image-1",
    prompt=prompt_imagen,
    size="1024x1024"
)

# Guardar y mostrar
image_base64 = result.data[0].b64_json
image_bytes = base64.b64decode(image_base64)

os.makedirs("../assets/images", exist_ok=True)
path = "../assets/images/breathing_guide.png"
with open(path, "wb") as f:
    f.write(image_bytes)

print(f"✅ Imagen generada y guardada en {path}")
display(Image(filename=path))
